In [2]:
import sqlite3
import numpy as np
import pandas as pd
from pathlib import Path
import sys

# --- Configuration ---
# Add the 'modules' directory to the system path so we can import
# (This assumes the notebook is in the root directory)
sys.path.append('modules')

# Path to the database (relative to this notebook in the root)
# We can use the default path from your new manager
DB_PATH = Path("../data/processed/database/carbon_footprint.sqlite")

# Name of the OLD table you want to inspect# ---

print(f"Attempting to connect to database at: {DB_PATH.resolve()}")

conn = None
# Ensure the DB file exists before connecting
if not DB_PATH.exists():
    print(f"\nERROR: Database file not found at {DB_PATH.resolve()}")
    print("Please ensure the path is correct and the database has been created.")
else:
    # Connect to the SQLite database
    conn = sqlite3.connect(DB_PATH.as_posix())

cursor = conn.cursor()

Attempting to connect to database at: C:\Users\felipeproenca\Documents\workspaces\personal\carbon-footprint\data\processed\database\carbon_footprint.sqlite


In [3]:
df = cursor.execute("drop table Avenida_Professor_Luciano_Gualberto_Sao_Paulo__30tons ")

In [ ]:
print("\n--- Column Names and Types (Schema) ---")
# To get column types, we can query PRAGMA table_info
info_df = pd.read_sql_query(f"PRAGMA table_info({OLD_TABLE_NAME})", conn)
print(info_df[['name', 'type', 'notnull', 'dflt_value', 'pk']].to_string())

# Use pandas to read the SQL query into a DataFrame
query = f"SELECT * FROM {OLD_TABLE_NAME} LIMIT 20"
print(f"\nExecuting query: {query}")

df = pd.read_sql_query(query, conn)

if df.empty:
    # Check if the table actually exists but is just empty
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (OLD_TABLE_NAME,))
    if cursor.fetchone():
        print(f"\nQuery successful, but the table '{OLD_TABLE_NAME}' is empty.")
    else:
            print(f"\nERROR: The table '{OLD_TABLE_NAME}' does not exist in the database.")
else:
    print(f"\n--- First {len(df)} Rows from {OLD_TABLE_NAME} ---")
    print(df.to_string()) # .to_string() gives better formatting

Attempting to connect to database at: C:\Users\felipeproenca\Documents\workspaces\personal\carbon-footprint\data\database\carbon_footprint.sqlite
Will try to select data from table: heatmap_runs

--- Column Names and Types (Schema) ---
                     name       type  notnull       dflt_value  pk
0               unique_id    INTEGER        0             None   1
1             origin_name       TEXT        1             None   0
2              origin_lat       REAL        0             None   0
3              origin_lon       REAL        0             None   0
4            destiny_name       TEXT        1             None   0
5             destiny_lat       REAL        0             None   0
6             destiny_lon       REAL        0             None   0
7   road_only_distance_km       REAL        0             None   0
8             cab_po_name       TEXT        0             None   0
9             cab_pd_name       TEXT        0             None   0
10    cab_road_o_to_po_km  

In [5]:
conn.cursor().execute("ALTER TABLE routes RENAME COLUMN destiny TO destiny_name")

In [14]:
try:
    conn = sqlite3.connect(DB_PATH.as_posix())

    # SQL query to select only the columns you requested (for the O->D leg)
    # We also filter out rows where the road distance is missing.
    query = f"""
    SELECT
        origin_name AS origin,
        origin_lat,
        origin_lon,
        destiny_name AS destiny,
        destiny_lat,
        destiny_lon,
        road_only_distance_km,
        is_hgv,
        insertion_timestamp
    FROM
        {OLD_TABLE_NAME}
    """
    
    df_road_only_legs = pd.read_sql_query(query, conn)
    
    # Rename columns to match the new schema / your request
    column_rename_map = {
        'road_only_distance_km': 'distance_km'
    }
    
    df_road_only_legs = df_road_only_legs.rename(columns=column_rename_map)

    print("\n--- DataFrame Schema (df.info()) ---")
    df_road_only_legs.info()
    
    print(f"\n--- First 5 Rows of O->D Legs DataFrame ---")
    print(df_road_only_legs.head().to_string())

except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")


--- DataFrame Schema (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   origin               169 non-null    object 
 1   origin_lat           169 non-null    float64
 2   origin_lon           169 non-null    float64
 3   destiny              169 non-null    object 
 4   destiny_lat          165 non-null    float64
 5   destiny_lon          165 non-null    float64
 6   distance_km          144 non-null    float64
 7   is_hgv               144 non-null    float64
 8   insertion_timestamp  169 non-null    object 
dtypes: float64(6), object(3)
memory usage: 12.0+ KB

--- First 5 Rows of O->D Legs DataFrame ---
                                                   origin  origin_lat  origin_lon                   destiny  destiny_lat  destiny_lon  distance_km  is_hgv  insertion_timestamp
0  Avenida Professor Luciano Gualb

In [15]:
pd_to_d = f"""
SELECT distinct
    CONCAT("Porto de ", cab_pd_name) as origin
    , (
        CASE
            WHEN cab_pd_name = 'Manaus' THEN -3.1567
            WHEN cab_pd_name = 'Suape' THEN -8.394046412845151
            WHEN cab_pd_name = 'São Sebastião' THEN -23.808154673463267
            WHEN cab_pd_name = 'Maceió' THEN -9.677
            WHEN cab_pd_name = 'Santarém' THEN -2.422
            ELSE NULL
        END
    ) AS origin_lat
    , (
        CASE
            WHEN cab_pd_name = 'Manaus' THEN -60.0079
            WHEN cab_pd_name = 'Suape' THEN -34.97428186936652
            WHEN cab_pd_name = 'São Sebastião' THEN -45.39974141587111
            WHEN cab_pd_name = 'Maceió' THEN -35.726
            WHEN cab_pd_name = 'Santarém' THEN -54.719
            ELSE NULL
        END
    ) AS origin_lon
    , destiny_NAME as destiny
    , destiny_lat
    , destiny_lon
    , cab_road_pd_to_d_km distance_km
    , is_hgv
    , insertion_timestamp
FROM
    {OLD_TABLE_NAME}
WHERE
    cab_pd_name IS NOT NULL
"""

df_pd_to_d = pd.read_sql_query(pd_to_d, conn)


print(df_pd_to_d.head().to_string())


            origin  origin_lat  origin_lon                      destiny  destiny_lat  destiny_lon  distance_km  is_hgv  insertion_timestamp
0  Porto de Manaus     -3.1567    -60.0079       Acrelandia, R., Brazil    -9.827780   -66.883330    1284.1071       1  2025-11-13 17:27:29
1  Porto de Manaus     -3.1567    -60.0079     Assis Brasil, R., Brazil   -10.935672   -69.560485    1708.9187       1  2025-11-13 17:27:29
2  Porto de Manaus     -3.1567    -60.0079           Bujari, R., Brazil    -9.827082   -67.948456    1427.4206       1  2025-11-13 17:27:36
3  Porto de Manaus     -3.1567    -60.0079         Capixaba, R., Brazil   -10.569186   -67.674444    1447.2721       1  2025-11-13 17:27:36
4  Porto de Manaus     -3.1567    -60.0079  Cruzeiro do Sul, R., Brazil    -7.611057   -72.682291    2041.1557       1  2025-11-13 17:27:36


In [16]:
distinct_pd = f"""
SELECT distinct
    cab_pd_name
FROM
    {OLD_TABLE_NAME}
WHERE
    cab_pd_name IS NOT NULL
"""

df_distinct_pd = pd.read_sql_query(distinct_pd, conn)


print(df_distinct_pd.head().to_string())


     cab_pd_name
0         Manaus
1  São Sebastião
2         Maceió
3          Suape
4       Santarém


In [17]:
history_df = pd.concat([df_road_only_legs, df_pd_to_d], ignore_index=True)

In [18]:
display(history_df)

,origin,origin_lat,origin_lon,destiny,destiny_lat,destiny_lon,distance_km,is_hgv,insertion_timestamp
0,"Avenida Professor Luciano Gualberto, São Paulo...",-23.558808,-46.730357,"Acrelandia, R., Brazil",-9.827780,-66.883330,3462.2200,1.0,2025-11-13 17:27:29
1,"Avenida Professor Luciano Gualberto, São Paulo...",-23.558808,-46.730357,"Assis Brasil, R., Brazil",-10.935672,-69.560485,3887.0316,1.0,2025-11-13 17:27:29
2,"Avenida Professor Luciano Gualberto, São Paulo...",-23.558808,-46.730357,"Brasiléia, R., Brazil",-10.983470,-68.773106,NaN,NaN,2025-11-13 17:27:36
3,"Avenida Professor Luciano Gualberto, São Paulo...",-23.558808,-46.730357,"Bujari, R., Brazil",-9.827082,-67.948456,3605.5334,1.0,2025-11-13 17:27:36
4,"Avenida Professor Luciano Gualberto, São Paulo...",-23.558808,-46.730357,"Capixaba, R., Brazil",-10.569186,-67.674444,3625.3849,1.0,2025-11-13 17:27:36
...,...,...,...,...,...,...,...,...,...
308,Porto de Manaus,-3.156700,-60.007900,"Manicoré, AM, Brazil",-5.820228,-61.286186,462.1137,1.0,2025-11-13 17:41:32
309,Porto de Manaus,-3.156700,-60.007900,"Maués, AM, Brazil",-3.392145,-57.711967,4394.7137,1.0,2025-11-13 17:41:41
310,Porto de Santarém,-2.422000,-54.719000,"Nhamundá, AM, Brazil",-2.190039,-56.714163,1360.0909,0.0,2025-11-13 17:41:47
311,Porto de Manaus,-3.156700,-60.007900,"Novo Airao, AM, Brazil",-2.620784,-60.943784,202.7805,0.0,2025-11-13 17:42:01


In [19]:
# history_df.to_sql('routes', conn, if_exists='append', index=False)

In [20]:
import sqlite3
import csv

# Database and table details
table_name = 'routes'
csv_file = 'routes.csv'

try:
    # Connect to SQLite database
    cursor = conn.cursor()

    # Execute SELECT query
    cursor.execute(f'SELECT * FROM {table_name}')

    # Open CSV file for writing
    with open(csv_file, 'w', newline='') as outfile:
        csv_writer = csv.writer(outfile)

        # Write header row (column names)
        header = [description[0] for description in cursor.description]
        csv_writer.writerow(header)

        # Write data rows
        for row in cursor:
            csv_writer.writerow(row)

    print(f"Data from '{table_name}' exported to '{csv_file}' successfully.")

except sqlite3.Error as e:
    print(f"SQLite error: {e}")
except IOError as e:
    print(f"File I/O error: {e}")
finally:
    if conn:
        conn.close()

Data from 'routes' exported to 'routes.csv' successfully.
